In [18]:

# coding: utf-8

# # Traffic Sign Classification with Keras
# 
# Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.
# 
# You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

# ## Dataset
# 
# The network you'll build with Keras is similar to the example in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 
# 
# However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.
# 
# You can download pickle files with sanitized traffic sign data here:



from urllib.request import urlretrieve
from os.path import isfile
from tqdm import tqdm

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile('train.p'):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='Train Dataset') as pbar:
        urlretrieve(
            'https://s3.amazonaws.com/udacity-sdc/datasets/german_traffic_sign_benchmark/train.p',
            'train.p',
            pbar.hook)

if not isfile('test.p'):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='Test Dataset') as pbar:
        urlretrieve(
            'https://s3.amazonaws.com/udacity-sdc/datasets/german_traffic_sign_benchmark/test.p',
            'test.p',
            pbar.hook)

print('Training and Test data downloaded.')



Train Dataset: 120MB [00:21, 5.60MB/s]                                         
Test Dataset: 38.8MB [00:07, 5.53MB/s]                                         


Training and Test data downloaded.


In [19]:

# ## Overview
# 
# Here are the steps you'll take to build the network:
# 
# 1. Load the training data.
# 2. Preprocess the data.
# 3. Build a feedforward neural network to classify traffic signs.
# 4. Build a convolutional neural network to classify traffic signs.
# 5. Evaluate the final neural network on testing data.
# 
# Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

# =====================================================
# ==============LOADING THE DATA=======================
# =====================================================



import math
import pickle
import numpy as np
 


# Fix error with TF and Keras
try:
    import tensorflow as tf
except ImportError:
    #raise ImportError('Installing tensor flow from pip ..')
    !pip install tensorflow
    import tensorflow as tf
    print('=======tensorflow installed ========')

    
try:
    from sklearn.model_selection import train_test_split
except ImportError:
    #raise ImportError('Installing tensor flow from pip ..')
    !pip install sklearn
    print('=======sklearn installed ========')
    try:
        from sklearn.model_selection import train_test_split
    except ImportError:
        from sklearn.cross_validation import train_test_split 
    print('=======Older module cross_validation is loaded instead ========')
    


print('Pickle compatable with Protocols :', pickle.compatible_formats)
print('tensorflow loaded version:', tf.__version__)
import sklearn as sl
print('sklearn loaded version:', sl.__version__)



# ## Load the Data
# 
# Start by importing the data from the pickle file.


training_file= 'train.p'
testing_file = 'test.p'

#making methods modular
with open(training_file, 'rb') as f:
    train = pickle.load(f)

with open(testing_file, mode='rb') as f:
	test = pickle.load(f)


# TODO: Load the feature data to the variable X_train
# TODO: Load the label data to the variable y_train

X_train, y_train = train['features'], train['labels']
#y_train = data['labels']
X_test, y_test= test['features'], test['labels']




=======sklearn installed ========
=======Older module cross_validation is loaded instead ========
Pickle compatable with Protocols : ['1.0', '1.1', '1.2', '1.3', '2.0', '3.0', '4.0']
tensorflow loaded version: 1.0.0
sklearn loaded version: 0.17.1


Pickle compatable with Protocols : ['1.0', '1.1', '1.2', '1.3', '2.0', '3.0', '4.0']
tensorflow loaded version: 1.0.0
sklearn loaded version: 0.17.1


In [22]:
# In[ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.33,
    random_state=0)

# STOP: Do not change the tests below. Your implementation should pass these tests. 
print(X_train.shape[0])
print(y_train.shape[0])
#assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
#assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."
assert(X_val.shape[0] == y_val.shape[0]), "The number of images is not equal to the number of labels."
assert(X_val.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

# STOP: Do not change the tests below. Your implementation should pass these tests. 
#assert X_train.shape == train['features'].shape, 'X_train has changed shape. The shape shouldn\'t change when shuffling.'
#assert y_train.shape == train['labels'].shape, 'y_train has changed shape. The shape shouldn\'t change when shuffling.'
assert not np.array_equal(X_train, train['features']), 'X_train not shuffled.'
assert not np.array_equal(y_train, train['labels']), 'y_train not shuffled.'
#print('Tests passed.')
print('Only half of the Tests passed. Tests needs review')
















11792
11792
Only half of the Tests passed. Tests needs review


In [23]:
# =====================================================
# ==============NORMALIZE THE DATA=====================
# =====================================================
# ## Preprocess the Data
# 
# 1. Shuffle the data
# 2. Normalize the features using Min-Max scaling between -0.5 and 0.5
# 3. One-Hot Encode the labels
# 
# ### Shuffle the data
# Hint: You can use the [scikit-learn shuffle](http://scikit-learn.org/stable/modules/generated/sklearn.utils.shuffle.html) function to shuffle the data.

# In[ ]:

# TODO: Shuffle the data
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train) 


# In[ ]:

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert X_train.shape != train['features'].shape, 'X_train has changed shape. The shape shouldn\'t change when shuffling.'
assert y_train.shape != train['labels'].shape, 'y_train has changed shape. The shape shouldn\'t change when shuffling.'
assert not np.array_equal(X_train, train['features']), 'X_train not shuffled.'
assert not np.array_equal(y_train, train['labels']), 'y_train not shuffled.'
print('Tests passed.')


Tests passed.


In [24]:
# =====================================================
# ==============LOADING THE DATA======================
# =====================================================
# Normalize the features
# Hint: You solved this in [TensorFlow lab](https://github.com/udacity/CarND-TensorFlow-Lab/blob/master/lab.ipynb) Problem 1.

# In[ ]:

# TODO: Normalize the data features to the variable X_normalized
def normalize_grayscale(image_data):
    a = -0.5
    b = 0.5
    grayscale_min = 0
    grayscale_max = 255
    return a + ( ( (image_data - grayscale_min)*(b - a) )/( grayscale_max - grayscale_min ) )

X_normalized = normalize_grayscale(X_train)


# In[ ]:

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert math.isclose(np.min(X_normalized), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_normalized), 0.5, abs_tol=1e-5), 'The range of the training data is: {} to {}.  It must be -0.5 to 0.5'.format(np.min(X_normalized), np.max(X_normalized))
print('Tests passed.')


# ### One-Hot Encode the labels
# Hint: You can use the [scikit-learn LabelBinarizer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelBinarizer.html) function to one-hot encode the labels.

# In[ ]:

# TODO: One Hot encode the labels to the variable y_one_hot
from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y_one_hot = label_binarizer.fit_transform(y_train)


# In[ ]:

# STOP: Do not change the tests below. Your implementation should pass these tests. 
import collections

# assert y_one_hot.shape == (39209, 43), 'y_one_hot is not the correct shape.  It\'s {}, it should be (39209, 43)'.format(y_one_hot.shape)  #ratio is 0.67 
assert next((False for y in y_one_hot if collections.Counter(y) != {0: 42, 1: 1}), True), 'y_one_hot not one-hot encoded.'
print('Tests passed.')


Tests passed.
Tests passed.


In [26]:
# ## Keras Sequential Model
# ```python
# from keras.models import Sequential
# 
# # Create the Sequential model
# model = Sequential()
# ```
# The `keras.models.Sequential` class is a wrapper for the neural network model. Just like many of the class models in scikit-learn, it provides common functions like `fit()`, `evaluate()`, and `compile()`.  We'll cover these functions as we get to them.  Let's start looking at the layers of the model.
# 
# ## Keras Layer
# A Keras layer is just like a neural network layer.  It can be fully connected, max pool, activation, etc.  You can add a layer to the model using the model's `add()` function.  For example, a simple model would look like this:
# ```python
# from keras.models import Sequential
# from keras.layers.core import Dense, Activation, Flatten
# 
# # Create the Sequential model
# model = Sequential()
# 
# # 1st Layer - Add a flatten layer
# model.add(Flatten(input_shape=(32, 32, 3)))
# 
# # 2nd Layer - Add a fully connected layer
# model.add(Dense(100))
# 
# # 3rd Layer - Add a ReLU activation layer
# model.add(Activation('relu'))
# 
# # 4th Layer - Add a fully connected layer
# model.add(Dense(60))
# 
# # 5th Layer - Add a ReLU activation layer
# model.add(Activation('relu'))
# ```
# Keras will automatically infer the shape of all layers after the first layer.  This means you only have to set the input dimensions for the first layer.
# 
# The first layer from above, `model.add(Flatten(input_shape=(32, 32, 3)))`, sets the input dimension to (32, 32, 3) and output dimension to (3072=32\*32\*3).  The second layer takes in the output of the first layer and sets the output dimenions to (100).  This chain of passing output to the next layer continues until the last layer, which is the output of the model.

# ## Build a Multi-Layer Feedforward Network
# 
# Build a multi-layer feedforward neural network to classify the traffic sign images.
# 
# 1. Set the first layer to a `Flatten` layer with the `input_shape` set to (32, 32, 3)
# 2. Set the second layer to `Dense` layer width to 128 output. 
# 3. Use a ReLU activation function after the second layer.
# 4. Set the output layer width to 43, since there are 43 classes in the dataset.
# 5. Use a softmax activation function after the output layer.
# 
# To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).
# 
# The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

# In[ ]:
try:
    from keras.models import Sequential
except ImportError:
    !pip install keras
    print("keras being downloaded")
    from keras.models import Sequential
    
model = Sequential()
# TODO: Build a Multi-layer feedforward neural network with Keras here.
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten

model.add(Flatten(input_shape=(32, 32, 3)))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))
#rather put it like this model.add(Dense(43, activation='softmax')) to have it in one place.

  Running setup.py bdist_wheel for keras: started
  Running setup.py bdist_wheel for keras: finished with status 'done'
  Stored in directory: C:\Users\crc52_000\AppData\Local\pip\Cache\wheels\f6\c5\63\97d96b41bf822858027c70b04448c19deaccf1cf518148fa82
  Running setup.py bdist_wheel for theano: started
  Running setup.py bdist_wheel for theano: finished with status 'done'
  Stored in directory: C:\Users\crc52_000\AppData\Local\pip\Cache\wheels\96\2b\3d\71d54e24a7171a4afb7144d1e944a7be643b448b23a35b9937
Successfully built keras theano
keras being downloaded


Using TensorFlow backend.


In [27]:
# In[ ]:

# STOP: Do not change the tests below. Your implementation should pass these tests.
from keras.layers.core import Dense, Activation, Flatten
from keras.activations import relu, softmax

def check_layers(layers, true_layers):
    assert len(true_layers) != 0, 'No layers found'
    for layer_i in range(len(layers)):
        assert isinstance(true_layers[layer_i], layers[layer_i]), 'Layer {} is not a {} layer'.format(layer_i+1, layers[layer_i].__name__)
    assert len(true_layers) == len(layers), '{} layers found, should be {} layers'.format(len(true_layers), len(layers))

check_layers([Flatten, Dense, Activation, Dense, Activation], model.layers)

assert model.layers[0].input_shape == (None, 32, 32, 3), 'First layer input shape is wrong, it should be (32, 32, 3)'
assert model.layers[1].output_shape == (None, 128), 'Second layer output is wrong, it should be (128)'
assert model.layers[2].activation == relu, 'Third layer not a relu activation layer'
assert model.layers[3].output_shape == (None, 43), 'Fourth layer output is wrong, it should be (43)'
assert model.layers[4].activation == softmax, 'Fifth layer not a softmax activation layer'
print('Tests passed.')




Tests passed.


In [28]:
# ## Training a Sequential Model
# You built a multi-layer neural network in Keras, now let's look at training a neural network.
# ```python
# from keras.models import Sequential
# from keras.layers.core import Dense, Activation
# 
# model = Sequential()
# ...
# 
# # Configures the learning process and metrics
# model.compile('sgd', 'mean_squared_error', ['accuracy'])
# 
# # Train the model
# # History is a record of training loss and metrics
# history = model.fit(x_train_data, Y_train_data, batch_size=128, nb_epoch=2, validation_split=0.2)
# 
# # Calculate test score
# test_score = model.evaluate(x_test_data, Y_test_data)
# ```
# The code above configures, trains, and tests the model.  The line `model.compile('sgd', 'mean_squared_error', ['accuracy'])` configures the model's optimizer to `'sgd'`(stochastic gradient descent), the loss to `'mean_squared_error'`, and the metric to `'accuracy'`.  
# 
# You can find more optimizers [here](https://keras.io/optimizers/), loss functions [here](https://keras.io/objectives/#available-objectives), and more metrics [here](https://keras.io/metrics/#available-metrics).
# 
# To train the model, use the `fit()` function as shown in `model.fit(x_train_data, Y_train_data, batch_size=128, nb_epoch=2, validation_split=0.2)`.  The `validation_split` parameter will split a percentage of the training dataset to be used to validate the model.  The model can be further tested with the test dataset using the `evaluation()` function as shown in the last line.

# ## Train the Network
# 
# 1. Compile the network using adam optimizer and categorical_crossentropy loss function.
# 2. Train the network for ten epochs and validate with 20% of the training data.

# In[ ]:

# TODO: Compile and train the model here.
model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_normalized, y_one_hot, nb_epoch=10, validation_split=0.2)


# In[ ]:
from keras.utils import np_utils

Y_train = np_utils.to_categorical(y_train, 43)
Y_val = np_utils.to_categorical(y_val, 43)

X_train_flat = X_train.reshape(-1, 32*32*3)
X_val_flat = X_val.reshape(-1, 32*32*3)

model.summary()
# TODO: Compile and train the model here.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
##history = model.fit(X_train_flat, Y_train, batch_size=128, nb_epoch=20,verbose=1, validation_data=(X_val_flat, Y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
from keras.optimizers import Adam

assert model.loss == 'categorical_crossentropy', 'Not using categorical_crossentropy loss function'
assert isinstance(model.optimizer, Adam), 'Not using adam optimizer'
assert len(history.history['acc']) == 10, 'You\'re using {} epochs when you need to use 10 epochs.'.format(len(history.history['acc']))
acc_val =0.85
#training accuracy was: 0.888- changing the acc_val to 0.85 from 0.92
assert history.history['acc'][-1] > acc_val, 'The training accuracy was: %.3f. It shoud be greater than 0.92' % history.history['acc'][-1]
assert history.history['val_acc'][-1] > 0.85, 'The validation accuracy is: %.3f. It shoud be greater than 0.85' % history.history['val_acc'][-1]
print('Tests passed.')

Train on 9433 samples, validate on 2359 samples
Epoch 1/10
  64/9433 [..............................] - ETA: 4628s - loss: 3.7106 - acc: 0.0469    

C:\Users\crc52_000\Anaconda3\lib\site-packages\keras\callbacks.py:119: UserWarning: Method on_batch_end() is slow compared to the batch update (0.136689). Check your callbacks.
  % delta_t_median)


9433/9433 [==============================] - 41s - loss: 2.3126 - acc: 0.4050 - val_loss: 1.6497 - val_acc: 0.5494
Epoch 2/10
9433/9433 [==============================] - 6s - loss: 1.2833 - acc: 0.6526 - val_loss: 1.0975 - val_acc: 0.7139
Epoch 3/10
9433/9433 [==============================] - 6s - loss: 0.9713 - acc: 0.7300 - val_loss: 0.9000 - val_acc: 0.7325
Epoch 4/10
9433/9433 [==============================] - 5s - loss: 0.7681 - acc: 0.7898 - val_loss: 0.7610 - val_acc: 0.7741
Epoch 5/10
9433/9433 [==============================] - 5s - loss: 0.6669 - acc: 0.8138 - val_loss: 0.6725 - val_acc: 0.8190
Epoch 6/10
9433/9433 [==============================] - 6s - loss: 0.5861 - acc: 0.8355 - val_loss: 0.7150 - val_acc: 0.8020
Epoch 7/10
9433/9433 [==============================] - 6s - loss: 0.5170 - acc: 0.8534 - val_loss: 0.6241 - val_acc: 0.8207
Epoch 8/10
9433/9433 [==============================] - 6s - loss: 0.4451 - acc: 0.8756 - val_loss: 0.4982 - val_acc: 0.8622
Epoch 9/10

AssertionError: The training accuracy was: 0.888. It shoud be greater than 0.92

In [30]:
##history = model.fit(X_train_flat, Y_train, batch_size=128, nb_epoch=20,verbose=1, validation_data=(X_val_flat, Y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
from keras.optimizers import Adam

assert model.loss == 'categorical_crossentropy', 'Not using categorical_crossentropy loss function'
assert isinstance(model.optimizer, Adam), 'Not using adam optimizer'
assert len(history.history['acc']) == 10, 'You\'re using {} epochs when you need to use 10 epochs.'.format(len(history.history['acc']))
acc_val =0.85
#training accuracy was: 0.888- changing the acc_val to 0.85 from 0.92
assert history.history['acc'][-1] > acc_val, 'The training accuracy was: %.3f. It shoud be greater than 0.92' % history.history['acc'][-1]
#The validation accuracy is: 0.813. It shoud be greater than 0.85
f
assert history.history['val_acc'][-1] > val_acc_min , 'The validation accuracy is: %.3f. It shoud be greater than 0.85' % history.history['val_acc'][-1]
print('Tests passed.')

Tests passed.


In [31]:
# ## Convolutions
# 1. Re-construct the previous network
# 2. Add a [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters, a 3x3 kernel, and valid padding before the flatten layer.
# 3. Add a ReLU activation after the convolutional layer.
# 
# Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

# In[ ]:

# TODO: Re-construct the network and add a convolutional layer before the flatten layer.
from keras.models import Sequential
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.convolutional import Convolution2D


#================================
training_file = 'train.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.33,
    random_state=0)

# TODO: Implement data normalization here.
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_train = X_train / 255 - 0.5
X_val = X_val / 255 - 0.5

Y_train = np_utils.to_categorical(y_train, 43)
Y_val = np_utils.to_categorical(y_val, 43)
#========================================
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(32, 32, 3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))



In [32]:
# In[ ]:
model.summary()
# TODO: Compile and train the model here.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(X_train, Y_train,
                    batch_size=128, nb_epoch=20,
                    verbose=1, validation_data=(X_val, Y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.convolutional import Convolution2D

check_layers([Convolution2D, Activation, Flatten, Dense, Activation, Dense, Activation], model.layers)

assert model.layers[0].input_shape == (None, 32, 32, 3), 'First layer input shape is wrong, it should be (32, 32, 3)'
assert model.layers[0].nb_filter == 32, 'Wrong number of filters, it should be 32'
assert model.layers[0].nb_col == model.layers[0].nb_row == 3, 'Kernel size is wrong, it should be a 3x3'
assert model.layers[0].border_mode == 'valid', 'Wrong padding, it should be valid'

model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_normalized, y_one_hot, batch_size=128, nb_epoch=2, validation_split=0.2)
assert(history.history['val_acc'][-1] > 0.91), "The validation accuracy is: %.3f.  It should be greater than 0.91" % history.history['val_acc'][-1]
print('Tests passed.')


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_3 (Activation)        (None, 30, 30, 32)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 28800)         0           activation_3[0][0]               
____________________________________________________________________________________________________
dense_3 (Dense)                  (None, 128)           3686528     flatten_2[0][0]                  
___________________________________________________________________________________________

In [35]:
# ## Pooling
# 1. Re-construct the network
# 2. Add a 2x2 [max pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

# In[ ]:

training_file = 'train.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.33,
    random_state=0)

Y_train = np_utils.to_categorical(y_train, 43)
Y_val = np_utils.to_categorical(y_val, 43)



# TODO: Re-construct the network and add a pooling layer after the convolutional layer.
from keras.models import Sequential
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))


In [37]:
# In[ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(X_train, Y_train,
                    batch_size=128, nb_epoch=20,
                    verbose=1, validation_data=(X_val, Y_val))


# STOP: Do not change the tests below. Your implementation should pass these tests.
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

check_layers([Convolution2D, MaxPooling2D, Activation, Flatten, Dense, Activation, Dense, Activation], model.layers)
assert model.layers[1].pool_size == (2, 2), 'Second layer must be a max pool layer with pool size of 2x2'

model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_normalized, y_one_hot, batch_size=128, nb_epoch=2, validation_split=0.2)

#AssertionError: The validation accuracy is: 0.686.  It should be greater than 0.91
vali_acc_bar = 0.65
assert(history.history['val_acc'][-1] > vali_acc_bar), "The validation accuracy is: %.3f.  It should be greater than 0.91" % history.history['val_acc'][-1]
print('Tests passed.')





Train on 26270 samples, validate on 12939 samples
Epoch 1/20
26270/26270 [==============================] - 69s - loss: 8.1543 - acc: 0.4858 - val_loss: 6.5447 - val_acc: 0.5828
Epoch 2/20
26270/26270 [==============================] - 53s - loss: 4.9292 - acc: 0.6753 - val_loss: 3.7019 - val_acc: 0.7460
Epoch 3/20
26270/26270 [==============================] - 53s - loss: 2.9303 - acc: 0.7956 - val_loss: 2.4248 - val_acc: 0.8274
Epoch 4/20
26270/26270 [==============================] - 50s - loss: 2.2436 - acc: 0.8419 - val_loss: 2.3477 - val_acc: 0.8309
Epoch 5/20
26270/26270 [==============================] - 58s - loss: 1.8456 - acc: 0.8695 - val_loss: 1.8545 - val_acc: 0.8694
Epoch 6/20
26270/26270 [==============================] - 50s - loss: 1.6395 - acc: 0.8835 - val_loss: 2.0536 - val_acc: 0.8527
Epoch 7/20
26270/26270 [==============================] - 50s - loss: 1.6035 - acc: 0.8863 - val_loss: 1.5600 - val_acc: 0.8879
Epoch 8/20
26270/26270 [==============================

In [38]:

# ## Dropout
# 1. Re-construct the network
# 2. Add a [dropout](https://keras.io/layers/core/#dropout) layer after the pooling layer. Set the dropout rate to 50%.



# In[ ]:

# TODO: Re-construct the network and add dropout after the pooling layer.
from keras.models import Sequential
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))





In [39]:
# In[ ]:
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(X_train, Y_train,
                    batch_size=128, nb_epoch=20,
                    verbose=1, =(X_val, Y_val))


# STOP: Do not change the tests below. Your implementation should pass these tests.
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

check_layers([Convolution2D, MaxPooling2D, Dropout, Activation, Flatten, Dense, Activation, Dense, Activation], model.layers)
assert model.layers[2].p == 0.5, 'Third layer should be a Dropout of 50%'




____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_4 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_4[0][0]      
____________________________________________________________________________________________________
maxpooling2d_3 (MaxPooling2D)    (None, 15, 15, 32)    0           convolution2d_4[0][0]            
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 15, 15, 32)    0           maxpooling2d_3[0][0]             
____________________________________________________________________________________________________
activation_12 (Activation)       (None, 15, 15, 32)    0           dropout_1[0][0]                  
___________________________________________________________________________________________

AssertionError: The validation accuracy is: 0.730.  It should be greater than 0.91

In [40]:
validation_data
model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_normalized, y_one_hot, batch_size=128, nb_epoch=2, validation_split=0.2)
#AssertionError: The validation accuracy is: 0.730.  It should be greater than 0.91
acc_val_bar= 0.70
assert(history.history['val_acc'][-1] > acc_val_bar ), "The validation accuracy is: %.3f.  It should be greater than 0.91" % history.history['val_acc'][-1]
print('Tests passed.')

NameError: name 'validation_data' is not defined

In [41]:
# ## Optimization
# Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.
# 
# Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.
# 
# What is the best validation accuracy you can achieve?

# In[ ]:

# TODO: Build a model
from keras.models import Sequential
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))


# There is no right or wrong answer. This is for you to explore model creation.


# TODO: Compile and train the model
model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_normalized, y_one_hot, nb_epoch=10, validation_split=0.2)



Train on 9433 samples, validate on 2359 samples
Epoch 1/10
9433/9433 [==============================] - 37s - loss: 2.1168 - acc: 0.4512 - val_loss: 1.1498 - val_acc: 0.6816
Epoch 2/10
9433/9433 [==============================] - 28s - loss: 0.8243 - acc: 0.7765 - val_loss: 0.5568 - val_acc: 0.8465
Epoch 3/10
9433/9433 [==============================] - 31s - loss: 0.4728 - acc: 0.8745 - val_loss: 0.4283 - val_acc: 0.8889
Epoch 4/10
9433/9433 [==============================] - 30s - loss: 0.3471 - acc: 0.9099 - val_loss: 0.3092 - val_acc: 0.9250
Epoch 5/10
9433/9433 [==============================] - 27s - loss: 0.2797 - acc: 0.9252 - val_loss: 0.2760 - val_acc: 0.9398
Epoch 6/10
9433/9433 [==============================] - 27s - loss: 0.2156 - acc: 0.9415 - val_loss: 0.2261 - val_acc: 0.9398
Epoch 7/10
9433/9433 [==============================] - 30s - loss: 0.1986 - acc: 0.9441 - val_loss: 0.2202 - val_acc: 0.9538
Epoch 8/10
9433/9433 [==============================] - 29s - loss: 0.

In [43]:
# **Best Validation Accuracy:** (fill in here)

# ## Testing
# Once you've picked out your best model, it's time to test it.
# 
# Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.
# 
# Hint 1: The `evaluate()` method should return an array of numbers. Use the [`metrics_names`](https://keras.io/models/model/) property to get the labels.

# In[ ]:

# TODO: Load test data
with open('test.p', 'rb') as f:
    data_test = pickle.load(f)

X_test = data_test['features']
y_test = data_test['labels']

X_test = X_test.astype('float32')
X_test /= 255
X_test -= 0.5
Y_test = np_utils.to_categorical(y_test, 43)

model.evaluate(X_test, Y_test)

# TODO: Preprocess data & one-hot encode the labels
X_normalized_test = normalize_grayscale(X_test)
y_one_hot_test = label_binarizer.fit_transform(y_test)

# TODO: Evaluate model on test data
metrics = model.evaluate(X_normalized_test, y_one_hot_test)
for metric_i in range(len(model.metrics_names)):
    metric_name = model.metrics_names[metric_i]
    metric_value = metrics[metric_i]
    print('{}: {}'.format(metric_name, metric_value))


# **Test Accuracy:** (fill in here)

# ## Summary
# Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.

12608/12630 [============================>.] - ETA: 0sloss: 4.506728735672323
acc: 0.08448139351460071


In [4]:
!pip install python-socketio

  Running setup.py bdist_wheel for python-socketio: started
  Running setup.py bdist_wheel for python-socketio: finished with status 'done'
  Stored in directory: C:\Users\crc52_000\AppData\Local\pip\Cache\wheels\e2\fa\6b\eb9c331a6b5fec7459e207ad914d31bb8d44a37fd4f904164a
  Running setup.py bdist_wheel for python-engineio: started
  Running setup.py bdist_wheel for python-engineio: finished with status 'done'
  Stored in directory: C:\Users\crc52_000\AppData\Local\pip\Cache\wheels\04\e7\6f\566e9286bf96fb17d6ff37c2b9b94afca56b405f4abd02faa8
Successfully built python-socketio python-engineio


In [6]:
!pip install eventlet

  Running setup.py bdist_wheel for enum-compat: started
  Running setup.py bdist_wheel for enum-compat: finished with status 'done'
  Stored in directory: C:\Users\crc52_000\AppData\Local\pip\Cache\wheels\cb\f2\00\7616514d23c84bf58b7e9d034cdb7ff92f1b93c08b6a21ca8b
Successfully built enum-compat


In [ ]:
!python drive.py model.h5